In [31]:
import cv2
import mediapipe as mp
import mediapipe.tasks.python.components.containers
import numpy as np
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from itertools import product
import math

In [ ]:

def calculate_length(p1 :tuple[int,int] , p2:tuple[int,int]) -> float:
    """
    두 점 사이의 거리를 계산합니다. (유클라디안 거리)
    :param p1: 첫 번째 점 (x, y)
    :param p2: 두 번째 점 (x, y)
    :return: 두 점 사이의 거리
    """
    x1, y1 = p1
    x2, y2 = p2
    return math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

def calculate_angle(p1:tuple[int,int], p2:tuple[int,int], p3:tuple[int,int]):
    """
    삼각형의 세 점 사이의 각도를 계산합니다.
    :param p1: 첫 번째 점 (x, y)
    :param p2: 두 번째 점 (x, y)
    :param p3: 세 번째 점 (x, y)
    :return: 삼각형의 세 점 사이의 각도 ∠p1p2p3
    """
    # 삼각형의 세 점 사이의 각도를 계산합니다.
    a = calculate_length(p1, p2)
    b = calculate_length(p2, p3)
    c = calculate_length(p3, p1)
    # 코사인 법칙을 사용하여 각도를 계산합니다.
    # ∠abc = arccos((a² + b² - c²) / 2ab)
    return math.degrees(math.acos((a ** 2 + b ** 2 - c ** 2) / (2 * a * b)))

In [45]:
# 1. 미디어파이프를 사용하여 비디오에서 스켈레톤 구조 추출하기
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

cap = cv2.VideoCapture('./videos/Standard.MOV')  # 비디오 파일 경로
fps = int(cap.get(cv2.CAP_PROP_FPS))
pose_data = []
# cnt = 0
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        # if cnt == 100:
        #     break
        # cnt += 1
        ret, frame = cap.read()
        if not ret:
            break
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        height, width, _ = image.shape
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
        
            pose_data.append([landmark.x * width for landmark in landmarks[11:17]]+[landmark.x *width for landmark in landmarks[23:29]]+[landmark.y * height for landmark in landmarks[11:17]]+[landmark.y * height for landmark in landmarks[23:29]])



cap.release()




In [46]:
pose_landmark_names = [
    'right_shoulder',
    'left_shoulder',
    'right_elbow',
    'left_elbow',
    'right_wrist',
    'left_wrist',
    'right_hip',
    'left_hip',
    'right_knee',
    'left_knee',
    'right_ankle',
    'left_ankle',
]
# 2. 시계열 데이터로 변환
pose_df = pd.DataFrame(pose_data)
pose_df.columns = [ x+'_x' for x in pose_landmark_names] + [ x+'_y' for x in pose_landmark_names]

In [48]:
pose_df.to_csv('./data/pose.csv', index=False)

In [47]:
print(pose_df.head())

   right_shoulder_x  left_shoulder_x  right_elbow_x  left_elbow_x  \
0        326.713257       335.876045     281.326351    297.142429   
1        326.521835       335.013657     281.462822    299.960995   
2        326.336670       333.814774     281.598148    299.712334   
3        326.423492       332.806549     281.877861    299.532833   
4        326.775627       331.547966     282.294273    297.512875   

   right_wrist_x  left_wrist_x  right_hip_x  left_hip_x  right_knee_x  \
0     285.878410    291.449547   384.745407  382.533302    332.834625   
1     284.959755    292.153320   387.052879  383.548698    332.837715   
2     284.768581    292.077789   388.151016  383.777428    332.851524   
3     284.715481    291.741219   388.992500  383.780823    332.853355   
4     284.705696    291.265316   389.641228  383.738480    332.853622   

   left_knee_x  ...  right_elbow_y  left_elbow_y  right_wrist_y  left_wrist_y  \
0   335.232658  ...     196.759515    195.174415     172.748852  